In [1]:
!pip install sodapy

    100% |████████████████████████████████| 829kB 824kB/s ta 0:00:01
  Running setup.py bdist_wheel for future ... done
  Stored in directory: /home/russell/.cache/pip/wheels/bf/c9/a3/c538d90ef17cf7823fa51fc701a7a7a910a80f6a405bf15b1a
Successfully built future
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [ ]:
# from https://github.com/socrata/dev.socrata.com/blob/39c6581986466edb5e7f72f5beea5ce69238f8de/snippets/pandas.py
import pandas as pd
from sodapy import socrata